# Phân tích dữ liệu thô

Import nhưng thư viện cần thiết

In [17]:
import numpy as np
import pandas as pd

Tạo các tiêu đề cho các cột của dữ liệu

In [18]:
cols = np.loadtxt("./arff_creator/zoo_cols.csv",delimiter=",", dtype=str)
cols

array(['animal-name', 'hair', 'feathers', 'eggs', 'milk', 'airborne',
       'aquatic', 'predator', 'toothed', 'backbone', 'breathes',
       'venomous', 'fins', 'legs', 'tail', 'domestic', 'catsize', 'class'],
      dtype='<U11')

Đọc file dữ liệu thô với tiêu đề tương ứng

In [19]:
zoo_df = pd.read_csv('./arff_creator/zoo.data', names=cols)
zoo_df.head(3)


,animal-name,hair,feathers,eggs,milk,airborne,aquatic,predator,toothed,backbone,breathes,venomous,fins,legs,tail,domestic,catsize,class
0,aardvark,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
1,antelope,1,0,0,1,0,0,0,1,1,1,0,0,4,1,0,1,1
2,bass,0,0,1,0,0,1,1,1,1,0,0,1,0,1,0,0,4


Hiển thị số lượng dòng (số đối tượng) và sốt lượng cột (thuộc tính và 1 cột của lớp):

In [20]:
num_rows, num_cols = zoo_df.shape
print(f'Số lượng dòng: {num_rows}\nSố lượng cột: {num_cols}')

Số lượng dòng: 101
Số lượng cột: 18


Kiểm tra các dòng dữ liệu có các dòng bị lặp không

In [21]:
have_duplicate_rows = zoo_df.duplicated().any()
print("Có dòng bị lặp: ", have_duplicate_rows)

Có dòng bị lặp:  False


Kiểm tra có giá trị animal name nào bị lặp không

In [22]:

have_duplicate_animal_names = zoo_df[cols[0]].duplicated().any()
print("Có animal name bị lặp: ", have_duplicate_animal_names)

Có animal name bị lặp:  True


Tỉ lệ giá trị thiếu và thống kê mô tả của từng cột

In [23]:
missing_ratio = zoo_df.isnull().sum()
missing_ratio = missing_ratio / num_rows
missing_ratio

animal-name    0.0
hair           0.0
feathers       0.0
eggs           0.0
milk           0.0
airborne       0.0
aquatic        0.0
predator       0.0
toothed        0.0
backbone       0.0
breathes       0.0
venomous       0.0
fins           0.0
legs           0.0
tail           0.0
domestic       0.0
catsize        0.0
class          0.0
dtype: float64

# Chuẩn hóa dữ liệu


Chuyển các giá trị 0/1 ở file dữ liệu thôi thành no/yes

Dựa vào mô tả dữ liệu Zoo Animal Classification ở https://www.kaggle.com/datasets/uciml/zoo-animal-classification, tôi biết được 7 lớp của dữ liệu Zoo bao gồm: Mammal, Bird, Reptile, Fish, Amphibian, Bug và Invertebrate

Tạo một file zoo_classes.csv để lưu trữ các lớp

In [24]:
# load cols and classes
classes = np.loadtxt("./arff_creator/zoo_classes.csv", delimiter=",", dtype=str)
classes.tolist()

['Mammal', 'Bird', 'Reptile', 'Fish', 'Amphibian', 'Bug', 'Invertebrate']

In [25]:
# change 0 to "no" and 1 to "yes" and fix class names
def fix_zoo_df(zoo_df):
    # change no to test
    for col in [col for col in cols[1:-1] if col != 'legs']:
        col_to_replace = zoo_df[col]

        col_to_replace.replace(to_replace=0, value='no', inplace=True)
        col_to_replace.replace(to_replace=1, value='yes', inplace=True)

        zoo_df[col] = col_to_replace

    for idx, animal_class in enumerate(classes):
        col_to_replace = zoo_df['class']
        col_to_replace.replace(to_replace=idx + 1, value=animal_class, inplace=True)

    return zoo_df

Kiểm tra dữ liệu đã chuẩn hóa

In [26]:
fix_zoo_df(zoo_df).head(3)

,animal-name,hair,feathers,eggs,milk,airborne,aquatic,predator,toothed,backbone,breathes,venomous,fins,legs,tail,domestic,catsize,class
0,aardvark,yes,no,no,yes,no,no,yes,yes,yes,yes,no,no,4,no,no,yes,Mammal
1,antelope,yes,no,no,yes,no,no,no,yes,yes,yes,no,no,4,yes,no,yes,Mammal
2,bass,no,no,yes,no,no,yes,yes,yes,yes,no,no,yes,0,yes,no,no,Fish


# Tạo phần dữ liệu

Tạo file arff thỏa yêu cầu đề bài bằng cách nối 3 phần explaination, relation và data


In [27]:
def create_arff(explanation_file, relation_file, data_file, output_file):
    filenames = [explanation_file, relation_file, data_file]
    with open(output_file, 'w') as outfile:
        for fname in filenames:
            with open(fname) as infile:
                for line in infile:
                    outfile.write(line)
                
                outfile.write("\n\n")
        outfile.write("%\n%\n%\n")

In [28]:

create_arff("./arff_creator/zoo_arff_explaination.txt",
                "./arff_creator/zoo_arff_relation.txt", 
                "./arff_creator/zoo_arff_data.txt", 
                "./arff/zoo.arff")

## Tạo file cần dự đoán ở dạng csv

In [30]:
predict_df = pd.read_csv("./predict.csv", names=cols)
predict_df.to_csv("./predict.csv", header=False, index=False)
predict_df

,animal-name,hair,feathers,eggs,milk,airborne,aquatic,predator,toothed,backbone,breathes,venomous,fins,legs,tail,domestic,catsize,class
NaN,animal-name,hair,feathers,eggs,milk,airborne,aquatic,predator,toothed,backbone,breathes,venomous,fins,legs,tail,domestic,catsize,class
0.0,NameIsSecret,yes,no,no,yes,no,no,no,yes,yes,yes,no,no,4,yes,no,yes,?
1.0,NameIsSecret,no,yes,yes,no,yes,no,no,no,yes,yes,no,no,2,yes,yes,no,?
2.0,NameIsSecret,no,no,yes,no,no,no,yes,yes,yes,yes,yes,no,0,yes,no,no,?
3.0,NameIsSecret,no,no,yes,no,no,yes,yes,yes,yes,no,no,yes,0,yes,no,no,?
4.0,NameIsSecret,no,no,yes,no,no,yes,yes,yes,yes,yes,no,no,4,yes,no,no,?
